# Usage
This notebook will bulk close SOAR cases in Google SecOps. This is useful for production planning where you may want to bulk remove test cases from testing

Note: This will close all cases in the Default environment by default. You can use the `case_name_match` setting to filter based upon a certain case name. If you'd like update the script, you can also manually set the `end_time` to only close cases up until a certain date.


In [ ]:
# @title Bulk close cases
api_key = "" # @param {"type":"string","placeholder":"11111-222-333-444-5555555"}
soar_hostname = "" # @param {"type":"string","placeholder":"mysoar.siemplify-soar.com"}
case_name_match = "" # @param {"type":"string","placeholder":""}
import requests
import json
from datetime import datetime

# Get the current time in UTC
now_utc = datetime.utcnow()

# Format the time as a string
end_time = now_utc.strftime("%Y-%m-%dT%H:%M:%S") + "Z"

headers = {
            'content-type': 'application/json',
            'Appkey' : api_key
            }
payload = {
            "startTime": "2023-07-10T00:00:00.000Z",
            "endTime": end_time,
            "isCaseClosed": False,
            "pageSize": 10000,
            "timeRangeFilter": 0,
            "sortBy": {
              "sortOrder": 0,
              "sortBy": "Id"
            }
          }
endpoint = 'https://' + soar_hostname + '/api/external/v1/search/CaseSearchEverything'
response = requests.post(url=endpoint, data=json.dumps(payload), headers=headers)

print(response.status_code)
json_results = response.json()
for i in json_results['results']:
    if case_name_match in i['title']:
      print(i['id'])
      del_payload = {
                    "caseId": i['id'],
                    "reason": "NotMalicious",
                    "rootCause": "Lab test",
                    "comment": "Closed with API automation",
                 }
      del_endpoint = 'https://' + soar_hostname + '/api/external/v1/cases/CloseCase'
      del_resp = requests.post(url=del_endpoint, data=json.dumps(del_payload), headers=headers)
      print(del_resp.text)


